In [7]:
from typing import List
from spacy.tokens import Doc, Span
from utils import load_models, print_clusters, print_comparison

In [2]:
predictor, nlp = load_models()

Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## The problem
AllenNLP coreference resolution models seems to find better clusters - numerous clusters that are usually more accurate than the ones found by Huggingface NeuralCoref model. However, the biggest problem lies in the next step - the step of replacing found mentions with the most meaningfull spans from each clusters (that we call the "heads"). We've found a couple of easy-to-fix problems which seem to lead to errors most often. Our ideas can be summed up as:
- not resolving coreferences in the clusters that doesn't contain any noun phrases (usually it comes down to the clusters composed only of pronouns),
- chosing the head of the cluster which is a noun phrase (isn't a pronoun),
- resolving only the inner span in the case of nested coreferent mentions.

We show all of our improvements by example - for more details please refer to our [third and last blog post]() that contain more details.

## Original AllenNLP impelemntation of the `replace_corefs` method

We extract the main "logic" into the separate function that will be used in our every method as we leave the core of AllenNLP's logic untouched. So as for now we will compare our solutions to the `original_replace_corefs` method implemented in AllenNLP `coref.py`.

In [3]:
def core_logic_part(document: Doc, coref: List[int], resolved: List[str], mention_span: Span):
    final_token = document[coref[1]]
    if final_token.tag_ in ["PRP$", "POS"]:
        resolved[coref[0]] = mention_span.text + "'s" + final_token.whitespace_
    else:
        resolved[coref[0]] = mention_span.text + final_token.whitespace_
    for i in range(coref[0] + 1, coref[1] + 1):
        resolved[i] = ""
    return resolved


def original_replace_corefs(document: Doc, clusters: List[List[List[int]]]) -> str:
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
        mention_span = document[mention_start:mention_end]

        for coref in cluster[1:]:
            core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

## Improvements
### Redundant clusters - lack of a meaningfull mention that could become the head
We completely ignore (we don't resove them at all) the clusters that doesn't contain any noun phrase.

In [10]:
def get_span_noun_indices(doc: Doc, cluster: List[List[int]]) -> List[int]:
    spans = [doc[span[0]:span[1]+1] for span in cluster]
    spans_pos = [[token.pos_ for token in span] for span in spans]
    span_noun_indices = [i for i, span_pos in enumerate(spans_pos)
        if any(pos in span_pos for pos in ['NOUN', 'PROPN'])]
    return span_noun_indices

def improved_replace_corefs(document, clusters):
    resolved = list(tok.text_with_ws for tok in document)

    for cluster in clusters:
        noun_indices = get_span_noun_indices(document, cluster)

        if noun_indices:
            mention_start, mention_end = cluster[0][0], cluster[0][1] + 1
            mention_span = document[mention_start:mention_end]

            for coref in cluster[1:]:
                core_logic_part(document, coref, resolved, mention_span)

    return "".join(resolved)

**Example**  
We want to take our code and create a game. Let's remind ourselves how to do that.  
- We --> We; our; 's; ourselves  
- create --> create; that

In [11]:
text = "We want to take our code and create a game. Let's remind ourselves how to do that."
_, _, _, document, clusters = predictor.predict(text).values()
doc = nlp(text)

In [12]:
print_comparison(original_replace_corefs(doc, clusters), improved_replace_corefs(doc, clusters))

AllenNLP original replace_corefs:
We want to take We's code and create a game. LetWe remind We how to do create.

Our improved replace_corefs:
We want to take our code and create a game. Let's remind ourselves how to do that.
